imports

In [1]:
import pandas as pd
import numpy as np
import pathlib
import os
from PIL import Image
import matplotlib.pyplot as plt
from characterization_ams.stats_engine import stats
from characterization_ams.emva import emva
from characterization_ams.standard_tests import ptc
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2

In [2]:
np.__version__


'1.20.3'

image location

In [3]:
illum_images = pathlib.Path(r'C:\Users\pbae.OFFICE\Documents\MEGAsync\ams\mira050illum_25_x16_nolens')
results_illum = illum_images/'results.csv'
dark_images = pathlib.Path(r'C:\Users\pbae.OFFICE\Documents\MEGAsync\ams\mira050dark_25')
results_dark = dark_images/'results.csv'

In [4]:
df_illum=pd.read_csv(results_illum)
df_dark_all=pd.read_csv(results_dark)

with pd.option_context('display.max_rows', 5,
                       'display.max_columns', 8,
                       'display.precision', 3,
                       'display.max_colwidth', 30
                       ):
    df_dark=df_dark_all[(df_dark_all["again"] == 16) & (df_dark_all["exp_us"] == 125)]
    display(df_dark)
    display(df_illum)

,bitmode,again,dgain,exp_us,filename,testcase,temperature,amount
8,10bit,16,1,125,10bit_again_16_dgain_1_exp...,dark_25,25,15


,bitmode,again,dgain,exp_us,filename,testcase,temperature,amount
0,10bit,16,1,100,10bit_again_16_dgain_1_exp...,illum_25_x16_nolens,25,5
1,10bit,16,1,120,10bit_again_16_dgain_1_exp...,illum_25_x16_nolens,25,5
...,...,...,...,...,...,...,...,...
68,10bit,16,1,760,10bit_again_16_dgain_1_exp...,illum_25_x16_nolens,25,5
69,10bit,16,1,780,10bit_again_16_dgain_1_exp...,illum_25_x16_nolens,25,5


merge both dataframes, so the first row is the dark img

In [5]:
df_both=df_dark.append(df_illum, ignore_index = True)
display(df_both)

,bitmode,again,dgain,exp_us,filename,testcase,temperature,amount
0,10bit,16,1,125,10bit_again_16_dgain_1_exp_us_ 125_dark_25__te...,dark_25,25,15
1,10bit,16,1,100,10bit_again_16_dgain_1_exp_us_ 100_illum_25_x1...,illum_25_x16_nolens,25,5
2,10bit,16,1,120,10bit_again_16_dgain_1_exp_us_ 120_illum_25_x1...,illum_25_x16_nolens,25,5
3,10bit,16,1,140,10bit_again_16_dgain_1_exp_us_ 140_illum_25_x1...,illum_25_x16_nolens,25,5
4,10bit,16,1,160,10bit_again_16_dgain_1_exp_us_ 160_illum_25_x1...,illum_25_x16_nolens,25,5
...,...,...,...,...,...,...,...,...
66,10bit,16,1,700,10bit_again_16_dgain_1_exp_us_ 700_illum_25_x1...,illum_25_x16_nolens,25,5
67,10bit,16,1,720,10bit_again_16_dgain_1_exp_us_ 720_illum_25_x1...,illum_25_x16_nolens,25,5
68,10bit,16,1,740,10bit_again_16_dgain_1_exp_us_ 740_illum_25_x1...,illum_25_x16_nolens,25,5
69,10bit,16,1,760,10bit_again_16_dgain_1_exp_us_ 760_illum_25_x1...,illum_25_x16_nolens,25,5


In [6]:
imnames_dark=df_dark['filename']
imnames_illum=df_illum['filename']
bitmode=df_illum['bitmode'][0]
bpp=int(bitmode.split('bit')[0])
print(bpp)


10


In [7]:
ptclist=[]
for imname in imnames_dark:
    images=[]
    for imname2 in dark_images.glob(f'./{imname}*.tiff'):
        im = Image.open(imname2)
        images.append(np.array(im))
    images=np.array(images)
    images=images/(2**(16-bpp))
    ptclist = [i for i in ptclist]
    ptclist.append(images)

for imname in imnames_illum:
    images=[]
    for imname2 in illum_images.glob(f'./{imname}*.tiff'):
        im = Image.open(imname2)
        images.append(np.array(im))
    images=np.array(images)
    images=images/(2**(16-bpp))
    ptclist = [i for i in ptclist]
    ptclist.append(images)


In [14]:
len(ptclist)


71

check if first image is dark

In [9]:
display(np.mean(ptclist[0]))

82.21148402777777

In [12]:
# df.rename(columns = {'mean':'meanold'}, inplace = True)
#statistics=ptc.get_stats(ptclist, df=df)
data, hist, summ=ptc.ptc(ptclist, df_both, exp_col='exp_us')
display(data)

c:\teamforge\characterization\trunk\characterization_ams\stats_engine\stats.py:86: RuntimeWarning: invalid value encountered in double_scalars
  var_noise['col_var']**0.5)
c:\teamforge\characterization\trunk\characterization_ams\stats_engine\stats.py:89: RuntimeWarning: invalid value encountered in double_scalars
  var_noise['col_var_temp']**0.5
c:\teamforge\characterization\trunk\characterization_ams\stats_engine\stats.py:92: RuntimeWarning: invalid value encountered in double_scalars
  var_noise['row_var']**0.5
c:\teamforge\characterization\trunk\characterization_ams\stats_engine\stats.py:95: RuntimeWarning: invalid value encountered in double_scalars
  var_noise['col_var_temp']**0.5
c:\teamforge\characterization\trunk\characterization_ams\stats_engine\stats.py:108: RuntimeWarning: invalid value encountered in double_scalars
  var_noise['mean'] * 100
c:\teamforge\characterization\trunk\characterization_ams\stats_engine\stats.py:111: RuntimeWarning: invalid value encountered in double

,Tot Var [DN^2],Col Var [DN^2],Row Var [DN^2],Pix Var [DN^2],Tot Temp Var [DN^2],Col Temp Var [DN^2],Row Temp Var [DN^2],Pix Temp Var [DN^2],Mean Signal [DN],CFPN Ratio,...,Linearity Fit [DN],Linearity Error [%],Linearity Error [DN],Responsivity Fit [DN],SNR [dB],SNR [ratio],SNR Ideal [dB],SNR Ideal [ratio],SNR Theoretical [dB],SNR Theoretical[ratio]
0,35.389875,4.871056,3.603741,26.960068,119.107657,2.807358,43.385939,73.036240,82.211484,4.944909,...,NaN,NaN,NaN,290.840589,NaN,NaN,NaN,NaN,NaN,NaN
1,173.729583,11.618606,13.716142,148.642883,605.799526,2.415980,47.770427,556.541850,345.413494,7.220835,...,262.731527,0.179074,0.470483,212.018598,20.844935,11.021653,21.495020,11.878210,21.317926,11.638481
2,173.729583,11.618606,13.716142,148.642883,605.799526,2.415980,47.770427,556.541850,345.413494,7.220835,...,262.731527,0.179074,0.470483,212.018598,20.844935,11.021653,21.495020,11.878210,21.317926,11.638481
3,205.406543,16.436692,17.160335,172.096703,708.802230,2.210537,52.912058,654.772288,397.261868,6.566823,...,315.116211,-0.020890,-0.065827,275.076191,21.624143,12.056109,22.275928,12.995602,22.127487,12.775396
4,205.406543,16.436692,17.160335,172.096703,708.802230,2.210537,52.912058,654.772288,397.261868,6.566823,...,315.116211,-0.020890,-0.065827,275.076191,21.624143,12.056109,22.275928,12.995602,22.127487,12.775396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1023.000000,NaN,...,NaN,NaN,NaN,NaN,25.916235,19.761129,27.027048,22.457034,26.976771,22.327421
67,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1023.000000,NaN,...,NaN,NaN,NaN,NaN,25.916235,19.761129,27.027048,22.457034,26.976771,22.327421
68,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1023.000000,NaN,...,NaN,NaN,NaN,NaN,25.916235,19.761129,27.027048,22.457034,26.976771,22.327421
69,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1023.000000,NaN,...,NaN,NaN,NaN,NaN,25.916235,19.761129,27.027048,22.457034,26.976771,22.327421


In [ ]:
display(hist)

In [ ]:
display(summ.T)

plot PTC

In [ ]:
fig = px.line(data, x='Signal - Dark [DN]', y=['Tot Temp Var - Tot Dark Temp Var [DN^2]','System Gain Fit [DN^2]'], 
              title='Photon transfer curve (PTC)')
fig.update_xaxes(rangemode="nonnegative")
fig.update_yaxes(rangemode="nonnegative")
fig.show()

In [ ]:
fig = px.line(data.iloc[1: 22, :], x='exp_us', y='Linearity Error [%]', 
              title='Linearity error')
fig.show()

In [ ]:
fig = px.line(data.iloc[1: , :], x='exp_us', y=['Signal - Dark [DN]', 
                                                            'Linearity Fit [DN]', 
                                                            'Responsivity Fit [DN]'], 
              title='Response curve')
fig.update_xaxes(rangemode="nonnegative")
fig.update_yaxes(rangemode="nonnegative")
fig.show()

In [17]:
stats_df = ptc.get_stats(images=ptclist,
df=df_both)

c:\teamforge\characterization\trunk\characterization_ams\stats_engine\stats.py:86: RuntimeWarning: invalid value encountered in double_scalars
  var_noise['col_var']**0.5)
c:\teamforge\characterization\trunk\characterization_ams\stats_engine\stats.py:89: RuntimeWarning: invalid value encountered in double_scalars
  var_noise['col_var_temp']**0.5
c:\teamforge\characterization\trunk\characterization_ams\stats_engine\stats.py:92: RuntimeWarning: invalid value encountered in double_scalars
  var_noise['row_var']**0.5
c:\teamforge\characterization\trunk\characterization_ams\stats_engine\stats.py:95: RuntimeWarning: invalid value encountered in double_scalars
  var_noise['col_var_temp']**0.5
c:\teamforge\characterization\trunk\characterization_ams\stats_engine\stats.py:108: RuntimeWarning: invalid value encountered in double_scalars
  var_noise['mean'] * 100
c:\teamforge\characterization\trunk\characterization_ams\stats_engine\stats.py:111: RuntimeWarning: invalid value encountered in double

In [19]:
stats_df

,Tot Var [DN^2],Col Var [DN^2],Row Var [DN^2],Pix Var [DN^2],Tot Temp Var [DN^2],Col Temp Var [DN^2],Row Temp Var [DN^2],Pix Temp Var [DN^2],Mean Signal [DN],CFPN Ratio,...,amount,Tot Var - Tot Dark Var [DN^2],Col Var - Col Dark Var [DN^2],Row Var - Row Dark Var [DN^2],Pix Var - Pix Dark Var [DN^2],Tot Temp Var - Tot Dark Temp Var [DN^2],Col Temp Var - Col Dark Temp Var [DN^2],Row Temp Var - Row Dark Temp Var [DN^2],Pix Temp Var - Pix Dark Temp Var [DN^2],Signal - Dark [DN]
0,38.400472,5.285434,3.910309,29.253546,119.107657,2.807358,43.385939,73.036240,82.211484,4.747112,...,15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,188.508662,12.606995,14.882967,161.287850,605.799526,2.415980,47.770427,556.541850,345.413494,6.932002,...,5,150.108190,7.321561,10.972658,132.034304,486.691869,-0.391378,4.384488,483.505610,263.202010
2,188.508662,12.606995,14.882967,161.287850,605.799526,2.415980,47.770427,556.541850,345.413494,6.932002,...,5,150.108190,7.321561,10.972658,132.034304,486.691869,-0.391378,4.384488,483.505610,263.202010
3,222.880363,17.834952,18.620155,186.736874,708.802230,2.210537,52.912058,654.772288,397.261868,6.304150,...,5,184.479891,12.549518,14.709846,157.483328,589.694572,-0.596821,9.526119,581.736048,315.050384
4,222.880363,17.834952,18.620155,186.736874,708.802230,2.210537,52.912058,654.772288,397.261868,6.304150,...,5,184.479891,12.549518,14.709846,157.483328,589.694572,-0.596821,9.526119,581.736048,315.050384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1023.000000,NaN,...,5,-38.400472,-5.285434,-3.910309,-29.253546,-119.107657,-2.807358,-43.385939,-73.036240,940.788516
67,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1023.000000,NaN,...,5,-38.400472,-5.285434,-3.910309,-29.253546,-119.107657,-2.807358,-43.385939,-73.036240,940.788516
68,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1023.000000,NaN,...,5,-38.400472,-5.285434,-3.910309,-29.253546,-119.107657,-2.807358,-43.385939,-73.036240,940.788516
69,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1023.000000,NaN,...,5,-38.400472,-5.285434,-3.910309,-29.253546,-119.107657,-2.807358,-43.385939,-73.036240,940.788516


In [21]:
ptclist[69]

array([[[1023., 1023., 1023., ..., 1023., 1023., 1023.],
        [1023., 1023., 1023., ..., 1023., 1023., 1023.],
        [1023., 1023., 1023., ..., 1023., 1023., 1023.],
        ...,
        [1023., 1023., 1023., ..., 1023., 1023., 1023.],
        [1023., 1023., 1023., ..., 1023., 1023., 1023.],
        [1023., 1023., 1023., ..., 1023., 1023., 1023.]],

       [[1023., 1023., 1023., ..., 1023., 1023., 1023.],
        [1023., 1023., 1023., ..., 1023., 1023., 1023.],
        [1023., 1023., 1023., ..., 1023., 1023., 1023.],
        ...,
        [1023., 1023., 1023., ..., 1023., 1023., 1023.],
        [1023., 1023., 1023., ..., 1023., 1023., 1023.],
        [1023., 1023., 1023., ..., 1023., 1023., 1023.]],

       [[1023., 1023., 1023., ..., 1023., 1023., 1023.],
        [1023., 1023., 1023., ..., 1023., 1023., 1023.],
        [1023., 1023., 1023., ..., 1023., 1023., 1023.],
        ...,
        [1023., 1023., 1023., ..., 1023., 1023., 1023.],
        [1023., 1023., 1023., ..., 1023., 102